In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [15]:
# Caminho para o diretório que contém as subpastas de classes
diretorio = 'imagens'

# Inicializa o gerador de dados com a opção de rescale e divisão de validação
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # reserva 20% das imagens para validação
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Cria o gerador de treino
train_generator = datagen.flow_from_directory(
    diretorio,
    target_size=(224, 224),  # redimensiona as imagens para 224x224
    batch_size=1,
    class_mode='binary',
    subset='training'  # define este conjunto como sendo de treinamento
)

# Cria o gerador de validação
validation_generator = datagen.flow_from_directory(
    diretorio,
    target_size=(224, 224),  # redimensiona as imagens para 224x224
    batch_size=1,
    class_mode='binary',
    subset='validation'  # define este conjunto como sendo de validação
)

Found 1117 images belonging to 2 classes.
Found 279 images belonging to 2 classes.


In [16]:
classes_train = train_generator.class_indices
classes_validation = validation_generator.class_indices

print("Classes no conjunto de treinamento:", classes_train)
print("Classes no conjunto de validação:", classes_validation)

Classes no conjunto de treinamento: {'corretas': 0, 'erradas': 1}
Classes no conjunto de validação: {'corretas': 0, 'erradas': 1}


In [17]:
# Contagem de imagens por classe no conjunto de treinamento
class_indices_train = train_generator.class_indices
labels_train = train_generator.labels
count_images_per_class_train = {class_name: labels_train.tolist().count(class_idx) for class_name, class_idx in class_indices_train.items()}

print("Quantidade de fotos por classe no conjunto de treinamento:", count_images_per_class_train)

# Contagem de imagens por classe no conjunto de validação
class_indices_validation = validation_generator.class_indices
labels_validation = validation_generator.labels
count_images_per_class_validation = {class_name: labels_validation.tolist().count(class_idx) for class_name, class_idx in class_indices_validation.items()}

print("Quantidade de fotos por classe no conjunto de validação:", count_images_per_class_validation)


Quantidade de fotos por classe no conjunto de treinamento: {'corretas': 1025, 'erradas': 92}
Quantidade de fotos por classe no conjunto de validação: {'corretas': 256, 'erradas': 23}


In [18]:
# Carregar a VGG16 como base do modelo, sem incluir as camadas superiores
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar as camadas do modelo base para não serem treinadas
for layer in base_model.layers:
    layer.trainable = False

# Adicionar novas camadas no topo para a tarefa específica
x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)  # Use dropout para reduzir o overfitting

predictions = layers.Dense(1, activation='sigmoid')(x)  # Use sigmoid para classificação binária

# Compilar o modelo
model = Model(inputs=base_model.input, outputs=predictions)
# model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

58889256/58889256 [==============================] - 6s 0us/step


ImportError: `load_weights` requires h5py package when loading weights from HDF5. Try installing h5py.

In [46]:
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=40,  # Depende do tamanho do seu conjunto de dados
    epochs=10,            # Ajuste conforme necessário
    validation_data=validation_generator,
    validation_steps=20,  # Depende do tamanho do seu conjunto de validação
    verbose=2)

Epoch 1/10
40/40 - 6s - 149ms/step - accuracy: 0.5750 - loss: 0.6344 - val_accuracy: 0.8500 - val_loss: 0.4712
Epoch 2/10
40/40 - 5s - 129ms/step - accuracy: 0.8750 - loss: 0.3848 - val_accuracy: 1.0000 - val_loss: 0.3774
Epoch 3/10
40/40 - 5s - 129ms/step - accuracy: 0.9000 - loss: 0.2914 - val_accuracy: 0.7000 - val_loss: 0.4032
Epoch 4/10
40/40 - 5s - 131ms/step - accuracy: 0.9250 - loss: 0.2399 - val_accuracy: 1.0000 - val_loss: 0.1845
Epoch 5/10
40/40 - 5s - 132ms/step - accuracy: 1.0000 - loss: 0.1279 - val_accuracy: 1.0000 - val_loss: 0.1824
Epoch 6/10
40/40 - 4s - 110ms/step - accuracy: 0.9750 - loss: 0.1568 - val_accuracy: 1.0000 - val_loss: 0.1144
Epoch 7/10


2024-03-22 17:18:05.507937: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


40/40 - 5s - 133ms/step - accuracy: 1.0000 - loss: 0.0647 - val_accuracy: 1.0000 - val_loss: 0.0965
Epoch 8/10
40/40 - 5s - 133ms/step - accuracy: 1.0000 - loss: 0.0599 - val_accuracy: 1.0000 - val_loss: 0.0708
Epoch 9/10
40/40 - 5s - 133ms/step - accuracy: 1.0000 - loss: 0.0530 - val_accuracy: 1.0000 - val_loss: 0.0739
Epoch 10/10
40/40 - 5s - 133ms/step - accuracy: 1.0000 - loss: 0.0446 - val_accuracy: 1.0000 - val_loss: 0.0548
